In [27]:
%load_ext autoreload
%autoreload 2

%pip install -r ../../requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


El sistema no puede encontrar la ruta especificada.


In [28]:
import sys

# Se agrega el path para poder importar los modulos locales
if "../" not in sys.path:
    sys.path.append("../")

import pandas as pd
import numpy as np
from utils import one_hot_encode

In [29]:
year = "2023"
flights = pd.read_parquet("../../resources/flights_clean.parquet")
flights = flights.astype({"Passengers": "int16", "Bookings": "int16"})
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Mes,Semana,DiaSemana,Hora,HourDuration
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229,157,10,42,4,11,2.750000
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197,109,7,27,1,0,4.000000
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200,142,6,26,1,15,0.666667
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162,90,2,6,5,8,1.166667
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183,125,9,36,4,17,0.916667


La variable real a predecir el modelo es la cantida de pasajeros de un vuelo individual

In [30]:
y = flights["Passengers"]
y.head()

0    229
1    197
2    200
3    162
4    183
Name: Passengers, dtype: int16

---

In [31]:
X = flights.drop(columns=["Passengers", "Flight_ID", "Bookings"])
X.head()

,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration
0,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,10,42,4,11,2.750000
1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,7,27,1,0,4.000000
2,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,6,26,1,15,0.666667
3,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,2,6,5,8,1.166667
4,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,9,36,4,17,0.916667


Transformación de los datos categoricos a utilizar a variables numéricos

In [32]:
for col, prefix in zip(
    [
        # "Aeronave",
        "DepartureStation",
        "ArrivalStation",
        "Destination_Type",
        "Origin_Type",
    ],
    [
        # "aeronave",
        "departure",
        "arrival",
        "destino",
        "origen",
    ],
):
    X = one_hot_encode(X, column=col, prefix=prefix)

X.head()

,Aeronave,STD,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,departure_ab,...,destino_ciudad_fronteriza,destino_ciudad_principal,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa
0,XA-VBY,2023-10-19 11:40:00,2023-10-19 14:25:00,240,10,42,4,11,2.750000,0,...,1,0,0,0,0,0,1,0,0,0
1,XA-VIX,2023-07-03 00:55:00,2023-07-03 04:55:00,186,7,27,1,0,4.000000,0,...,0,0,0,1,0,1,0,0,0,0
2,XA-VBW,2023-06-26 15:15:00,2023-06-26 15:55:00,220,6,26,1,15,0.666667,0,...,0,0,0,1,0,0,1,0,0,0
3,XA-VAP,2023-02-10 08:40:00,2023-02-10 09:50:00,186,2,6,5,8,1.166667,0,...,0,0,0,0,1,0,1,0,0,0
4,XA-VBY,2023-09-07 17:10:00,2023-09-07 18:05:00,240,9,36,4,17,0.916667,0,...,0,0,0,0,1,0,0,0,1,0


In [33]:
X["Dia"] = X["STD"].dt.day_of_year
X = X.drop(columns=["STD"])
X.head()

,Aeronave,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,departure_ab,departure_ac,...,destino_ciudad_principal,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia
0,XA-VBY,2023-10-19 14:25:00,240,10,42,4,11,2.750000,0,0,...,0,0,0,0,0,1,0,0,0,292
1,XA-VIX,2023-07-03 04:55:00,186,7,27,1,0,4.000000,0,0,...,0,0,1,0,1,0,0,0,0,184
2,XA-VBW,2023-06-26 15:55:00,220,6,26,1,15,0.666667,0,0,...,0,0,1,0,0,1,0,0,0,177
3,XA-VAP,2023-02-10 09:50:00,186,2,6,5,8,1.166667,0,0,...,0,0,0,1,0,1,0,0,0,41
4,XA-VBY,2023-09-07 18:05:00,240,9,36,4,17,0.916667,0,0,...,0,0,0,1,0,0,0,1,0,250


---

Se obtienen esta fecha del [calendario escolar de la SEP](https://calendarioescolar.sep.gob.mx)

In [34]:
dias_feriado = [
    1,  # Año Nuevo
    36,
    78,
    122,
    136,
    260,
    323,
    360,
    365,  # Año Nuevo
]

X["DiasAFeriadoCercano"] = X["Dia"].apply(
    lambda x: min([abs(x - feriado) for feriado in dias_feriado])
)

X.head()

,Aeronave,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,departure_ab,departure_ac,...,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia,DiasAFeriadoCercano
0,XA-VBY,2023-10-19 14:25:00,240,10,42,4,11,2.750000,0,0,...,0,0,0,0,1,0,0,0,292,31
1,XA-VIX,2023-07-03 04:55:00,186,7,27,1,0,4.000000,0,0,...,0,1,0,1,0,0,0,0,184,48
2,XA-VBW,2023-06-26 15:55:00,220,6,26,1,15,0.666667,0,0,...,0,1,0,0,1,0,0,0,177,41
3,XA-VAP,2023-02-10 09:50:00,186,2,6,5,8,1.166667,0,0,...,0,0,1,0,1,0,0,0,41,5
4,XA-VBY,2023-09-07 18:05:00,240,9,36,4,17,0.916667,0,0,...,0,0,1,0,0,0,1,0,250,10


In [35]:
# Dias entre 15 de julio y 28 de agosto
vacaciones_verano = (X["Dia"] >= 197) & (X["Dia"] <= 241)
# Dias entre 18 de diciembre y 5 de enero
vacaciones_invierno = (X["Dia"] >= 353) & (X["Dia"] <= 5)
# Dias entre 25 de marzo y 5 de abril
vacaciones_diatrabajo = (X["Dia"] >= 85) & (X["Dia"] <= 96)

X["VacacionesEscolares"] = np.where(
    (vacaciones_verano) | (vacaciones_invierno) | (vacaciones_diatrabajo), 1, 0
)
X.head()

,Aeronave,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,departure_ab,departure_ac,...,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia,DiasAFeriadoCercano,VacacionesEscolares
0,XA-VBY,2023-10-19 14:25:00,240,10,42,4,11,2.750000,0,0,...,0,0,0,1,0,0,0,292,31,0
1,XA-VIX,2023-07-03 04:55:00,186,7,27,1,0,4.000000,0,0,...,1,0,1,0,0,0,0,184,48,0
2,XA-VBW,2023-06-26 15:55:00,220,6,26,1,15,0.666667,0,0,...,1,0,0,1,0,0,0,177,41,0
3,XA-VAP,2023-02-10 09:50:00,186,2,6,5,8,1.166667,0,0,...,0,1,0,1,0,0,0,41,5,0
4,XA-VBY,2023-09-07 18:05:00,240,9,36,4,17,0.916667,0,0,...,0,1,0,0,0,1,0,250,10,0


In [36]:
X = X.drop(columns=["STA", "Aeronave"])

In [37]:
X.to_parquet(f"data/variables.parquet", index=False)